In [158]:
#pip install nom pour installer 
import pandas as pd

In [159]:
fichier = pd.read_csv("C:/Users/fanny/OneDrive/Bureau/Cours/projet_ia/Besoin_Client_1/export_IA.csv")

In [160]:
#affiche les premieres lignes du fichier 
print(fichier.head())

    id       MMSI          BaseDateTime       LAT       LON   SOG    COG  \
0   82  636017833  2023-05-29T00:00:06Z  29.07019 -89.29958  13.4  227.6   
1   91  367477280  2023-05-29T00:00:06Z  29.25456 -89.97235   0.0  267.4   
2  119  366954420  2023-05-29T00:00:10Z  27.77637 -82.63027   0.0  265.3   
3  222  367670000  2023-05-29T00:00:05Z  29.13941 -90.20319   0.0  165.1   
4  255  255805583  2023-05-29T00:00:02Z  28.86110 -95.15923   0.1   39.9   

   Heading        VesselName         IMO CallSign  VesselType  Status  Length  \
0      227       STOLT LOTUS  IMO9617648    D5MX5          80       0     183   
1      511              HERO  IMO8964147  WDH2114          60       0      47   
2      294  PROVINCETOWN III  IMO9329394  WDB8185          60       0      30   
3      340    HARVEY AMERICA  IMO9688477     KHAI          70       0      92   
4       64        CHEM SEA 1  IMO9731729     CQTD          80       1     160   

   Width  Draft  Cargo TransceiverClass  
0     28   10.

In [161]:
#sélection des colonnes utiles 
colonnes_interet = ['COG','LAT','LON']
# COG: direction reelle(influencé par des facteurs),   heading: orientation théorique du bateaux 
fichier_ia = fichier[colonnes_interet].copy()#on crée un nouveau tableau contenant que les colonnes selectionnés pour pas modifier le fichier original 
print(fichier_ia.head())

     COG       LAT       LON
0  227.6  29.07019 -89.29958
1  267.4  29.25456 -89.97235
2  265.3  27.77637 -82.63027
3  165.1  29.13941 -90.20319
4   39.9  28.86110 -95.15923


In [162]:
print(fichier_ia.dtypes)#pour savoir le type de chaque colonne 
#tout est deja en numerique 

COG    float64
LAT    float64
LON    float64
dtype: object


In [165]:
import numpy as np

 # conversion en radians
fichier_ia['COG_rad'] = np.deg2rad(fichier_ia['COG']) 
print(fichier_ia.head())

#conversion en cartesienne 
fichier_ia['x'] = np.cos(fichier_ia['COG_rad'])
fichier_ia['y'] = np.sin(fichier_ia['COG_rad'])
print(fichier_ia.head())

     COG       LAT       LON   COG_rad
0  227.6  29.07019 -89.29958  3.972369
1  267.4  29.25456 -89.97235  4.667010
2  265.3  27.77637 -82.63027  4.630359
3  165.1  29.13941 -90.20319  2.881539
4   39.9  28.86110 -95.15923  0.696386
     COG       LAT       LON   COG_rad         x         y
0  227.6  29.07019 -89.29958  3.972369 -0.674302 -0.738455
1  267.4  29.25456 -89.97235  4.667010 -0.045363 -0.998971
2  265.3  27.77637 -82.63027  4.630359 -0.081939 -0.996637
3  165.1  29.13941 -90.20319  2.881539 -0.966376  0.257133
4   39.9  28.86110 -95.15923  0.696386  0.767165  0.641450


In [ ]:
#on choisi la methode k means 
# # KMeans est un algorithme de clustering nan supervisé
#on choisi le nombre de custers k
#l'algorithme va positionne nos k clusters au hasard (centroides) 
#il regarde quel navire est le plus proche de quel centroide puis regroupe les point (navire) au centroide le plus proche
#il recalcule la position du centroide en faisant la moyenne de chaque donnees dans le cluster(donc d variables= d dimensions)
#il recommence jusqu’à ce que les groupes ne changent plus


# métode kmeans

### k=8

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pickle

donnees = fichier_ia[['x', 'y']].head(1000)

k = 8

#normalisation
scaler = StandardScaler()
donnees_norm = scaler.fit_transform(donnees)

#clustering
kmeans = KMeans(n_clusters=k, random_state=50)#on creer le modele 
kmeans.fit(donnees_norm)#on applique le modele cad onc cherche le centroide 
nums_clusters = kmeans.predict(donnees_norm)
# num_cluster est une liste de 1000 entiers représentant les groupes auxquels chaque point appartient
#predict attribue un numéro de cluster à chaque point de données

#creer une nouvelle colonne 'cluster_kmeans' qui contient les numeros de clusters 
fichier_ia.loc[donnees.index, 'cluster_kmeans'] = nums_clusters 

#enregistrer le pkl
with open('scaler_kmeans_direction.pkl', 'wb') as f:
    pickle.dump((scaler, kmeans), f)

print("Modèle et scaler enregistrés dans 'scaler_kmeans_direction.pkl'")

Modèle et scaler enregistrés dans 'scaler_kmeans_direction.pkl'


In [169]:
#calcul de l'indice de silhouette
from sklearn.metrics import silhouette_score

sil = silhouette_score(donnees, nums_clusters)
sil=round(sil,3)
print("indice de Calinski-Harabasz pour k=",k, ":",sil)


indice de Calinski-Harabasz pour k= 8 : 0.574


| Score de silhouette | Interprétation                                                  |
| ------------------- | --------------------------------------------------------------- |
| **0.71 – 1.00**     | Clustering excellent : très bonne séparation                 |
| **0.51 – 0.70**     | Bon clustering                                                |
| **0.26 – 0.50**     | Clustering moyen : un peu de mélange entre clusters          |
| **0.00 – 0.25**     | Clustering faible : clusters mal séparés, confusion possible |
| **< 0**             | Mauvais clustering : les points sont mal attribués            |


l'indice de silhouette evalue :
- La compacité du cluster (à quel point les points d'un meme cluster sont proche).
- La séparation avec les autres clusters (à quel point les clusters sont loin entre eux).

plus l'indice de silhouette est proche de 1 meilleur sera notre clutering

In [170]:
# 4. Calculer l’indice de Calinski-Harabasz
from sklearn.metrics import calinski_harabasz_score

ch = calinski_harabasz_score(donnees, nums_clusters)
ch=round(ch,3)
print("indice de Calinski-Harabasz pour k=",k, ":",ch)


indice de Calinski-Harabasz pour k= 8 : 3225.067


| Score CH       | Interprétation                          |
| -------------- | --------------------------------------- |
| **> 1000**     |  Excellente séparation des clusters   |
| **300 – 1000** |  Bonne séparation                      |
| **100 – 300**  |  Moyenne séparation                   |
| **< 100**      |  Mauvaise séparation – clusters flous |


plus l'indice de Calinski-Harabasz est grand meilleur sera notre clustering

In [171]:
#calcul de l'indice Davies-Bouldin
from sklearn.metrics import davies_bouldin_score

db = davies_bouldin_score(donnees, nums_clusters)
db=round(db,3)
print("indice de Davies-Bouldin pour k=",k, ":",db)


indice de Davies-Bouldin pour k= 8 : 0.495


| Score DB     | Interprétation                           |
| ------------- | ---------------------------------------- |
| **< 0.5**     |  Excellente séparation                 |
| **0.5 – 1.0** |  Bonne séparation                       |
| **> 1.0**     |  Clusters se chevauchent ou sont flous |


plus l'indice de Davies-Bouldin se rapproche de 0 meilleur sera notre clustering 

### k=4

In [185]:
donnees = fichier_ia[['x', 'y']].head(1000)

k = 4

#normalisation
scaler = StandardScaler()
donnees_norm = scaler.fit_transform(donnees)

#clustering
kmeans = KMeans(n_clusters=k, random_state=50)
kmeans.fit(donnees_norm)#fit: cherche les centres des groupes
nums_clusters = kmeans.predict(donnees_norm)
# num_cluster est une liste de 1000 entiers représentant les groupes auxquels chaque point appartient
#predict attribue un numéro de cluster à chaque point de données

#creer une nouvelle colonne 'cluster_kmeans' qui contient les numeros de clusters 
fichier_ia.loc[donnees.index, 'cluster_kmeans'] = nums_clusters 

In [186]:
#calcul de l'indice de silhouette
from sklearn.metrics import silhouette_score

sil = silhouette_score(donnees, nums_clusters)
sil=round(sil,3)
print("indice de Calinski-Harabasz pour k=",k, ":",sil)

indice de Calinski-Harabasz pour k= 4 : 0.54


In [187]:
# 4. Calculer l’indice de Calinski-Harabasz
from sklearn.metrics import calinski_harabasz_score

ch = calinski_harabasz_score(donnees, nums_clusters)
ch=round(ch,3)
print("indice de Calinski-Harabasz pour k=",k, ":",ch)

indice de Calinski-Harabasz pour k= 4 : 1662.041


In [189]:
#calcul de l'indice Davies-Bouldin
from sklearn.metrics import davies_bouldin_score

db = davies_bouldin_score(donnees, nums_clusters)
db=round(db,3)
print("indice de Davies-Bouldin pour k=",k, ":",db)

indice de Davies-Bouldin pour k= 4 : 0.61


conclusion: 
- tous les indices donnent des résultats corrects mais la qualité du clustering peu être amélioré
- on obtiens globalement une meilleure qualité de clustering pour k=8

# méthode du clustering hiérarchique 

In [172]:
#le clustering hierarchique commence par considérer chaque point comme un cluster tout seul, 
#puis fusionne les plus proches petit à petit, jusqu’à ce qu’il reste le nombre de groupes que tu veux.

from sklearn.cluster import AgglomerativeClustering
k=8
donnees = fichier_ia[['x', 'y']].head(1000)

clustering = AgglomerativeClustering(n_clusters=k)
nums_clusters = clustering.fit_predict(donnees)

fichier_ia.loc[donnees.index, 'cluster_agglom'] = nums_clusters



In [173]:
#calcul de l'indice de silhouette
from sklearn.metrics import silhouette_score

sil = silhouette_score(donnees, nums_clusters)
sil=round(sil,3)
print("indice de silouhette k=",k, ":",sil)

indice de silouhette k= 8 : 0.556


In [174]:
# 4. Calculer l’indice de Calinski-Harabasz
from sklearn.metrics import calinski_harabasz_score

ch = calinski_harabasz_score(donnees, nums_clusters)
ch=round(ch,3)
print("indice de Calinski-Harabasz pour k=",k, ":",ch)


indice de Calinski-Harabasz pour k= 8 : 2921.068


In [175]:
#calcul de l'indice Davies-Bouldin
from sklearn.metrics import davies_bouldin_score

db = davies_bouldin_score(donnees, nums_clusters)
db=round(db,3)
print("indice de Davies-Bouldin pour k=",k, ":",db)

indice de Davies-Bouldin pour k= 8 : 0.518


In [188]:
#calcul de l'indice Davies-Bouldin
from sklearn.metrics import davies_bouldin_score

db = davies_bouldin_score(donnees, nums_clusters)
db=round(db,3)
print("indice de Davies-Bouldin pour k=",k, ":",db)

indice de Davies-Bouldin pour k= 4 : 0.61


conclusion: 
donne des resultats similaire au kmeans

# méthode hdbscan

In [176]:
import hdbscan
import numpy as np

taille_echantillon=1000
k=4
donnees = fichier_ia[['x', 'y']].head(taille_echantillon)
min_taille_clusters = int(taille_echantillon/k)
hdb = hdbscan.HDBSCAN(min_cluster_size=min_taille_clusters)  
hdb.fit(donnees)

#récupération des labels des clusters (−1 = bruit, i.e. non-clusterisé)
nums_clusters = hdb.labels_

# Ajout d'une colonne avec le numéro du cluster
fichier_ia.loc[donnees.index, 'cluster_hdbscan'] = nums_clusters


#pour savoir le nombre de clusters
n_clusters = len(np.unique(nums_clusters[nums_clusters != -1]))
print("Nombre de clusters détectés :", n_clusters)
clusters_supp = np.sum(nums_clusters == -1)
print(clusters_supp)


Nombre de clusters détectés : 2
305


C:\Users\fanny\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

C:\Users\fanny\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [178]:
angles = (np.degrees(np.arctan2(donnees['y'], donnees['x']))) % 360

def conversion1(angle_deg):
    if (angle_deg >= 315 or angle_deg < 45):
        return "Nord"
    elif 45 <= angle_deg < 135:
        return "Est"
    elif 135 <= angle_deg < 225:
        return "Sud"
    elif 225 <= angle_deg < 315:
        return "Ouest"

donnees['direction_brute'] = angles.apply(conversion1)
print(donnees['direction_brute'].value_counts())

direction_brute
Ouest    359
Est      268
Nord     208
Sud      165
Name: count, dtype: int64


avec hdbscan on ne peut pas choisir le nombre de clusters qu'on veut or dans notre cas pour les directions on veut soit 4 clusters (nord, sud, est, ouest) soit 8 clusters (nord, sud, est, ouest, nord-est, nord-ouest, sud-est, sud-ouest). Si on se retrouve avec un nombre de clusters differents de ceux si ca n'a pas beaucoup d'interet. On peut néanmoins influencer le nombre de clusters avec la taille minimum d'un cluster, en effet on peut ce dire que il y aura un peu près le meme nombre de points dans chaque cluster. 
On peut voir que il suffit de peu de difference de nombres de points entre les directions pour que cette technique ne fonctionne pas (ici on a pris un echantillon de 1000 données donc le nombre minimal de point dans un cluster est de 250 mais on peu voir qu'il n'y a que les clusters est et ouest qui fonctionne)

# Représentation graphique avec la métode KMEANS

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

donnees = fichier_ia[['LAT', 'LON', 'x', 'y', 'cluster_kmeans']].dropna()
donnees['cluster_kmeans'] = donnees['cluster_kmeans'].astype(int)

#conversion angle en direction
def angle_vers_direction(angle_deg):
    if angle_deg >= 337.5 or angle_deg < 22.5:
        return 'Nord'
    elif 22.5 <= angle_deg < 67.5:
        return 'Nord-Est'
    elif 67.5 <= angle_deg < 112.5:
        return 'Est'
    elif 112.5 <= angle_deg < 157.5:
        return 'Sud-Est'
    elif 157.5 <= angle_deg < 202.5:
        return 'Sud'
    elif 202.5 <= angle_deg < 247.5:
        return 'Sud-Ouest'
    elif 247.5 <= angle_deg < 292.5:
        return 'Ouest'
    elif 292.5 <= angle_deg < 337.5:
        return 'Nord-Ouest'
    else:
        return 'Inconnu'

#on calcul la direction moyenne pour chaque cluster
cluster_direction_map = {}#initialisation du dictionnaire 
for cluster_id in sorted(donnees['cluster_kmeans'].unique()):#on trie dans l'ordre croissant tous les clusters trouvé
    points_cluster = donnees[donnees['cluster_kmeans'] == cluster_id]# renvoie true si la colonne cluster_kmeans contient cluster_id
    x_moy = points_cluster['x'].mean()
    y_moy = points_cluster['y'].mean()
    angle = (np.degrees(np.arctan2(y_moy, x_moy))) % 360#On convertit le centroïde (x_moy, y_moy) en angle en degrés
    direction = angle_vers_direction(angle)
    cluster_direction_map[cluster_id] = direction#ajoute au dictionnaire numero_cluster=dircetion

print(cluster_direction_map)
# on associe chaque point à sa direction
donnees['direction'] = donnees['cluster_kmeans'].map(cluster_direction_map)#.map() pour remplacer chaque numéro de cluster par son nom de direction.

#carte
fig = px.scatter_mapbox(
    donnees,
    lat="LAT",
    lon="LON",
    color="direction", 
    zoom=4,
    mapbox_style="open-street-map",#definir le fond de carte 
    title="Navires regroupés par direction de navigation avec la methode kmeans"
)

fig.show()

{0: 'Nord-Ouest', 1: 'Sud-Est', 2: 'Sud-Ouest', 3: 'Sud', 4: 'Nord-Est', 5: 'Est', 6: 'Ouest', 7: 'Nord'}


C:\Users\fanny\AppData\Local\Temp\ipykernel_6344\587201777.py:44: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



### montrons que la methode hdbsacan fonctionne avec d'autres variables ici par exemple la vitesse SOG

In [155]:
import pandas as pd
import numpy as np
import hdbscan

#on selectionne les colonnes utiles
colonnes_interet = ['SOG']
fichier_ia = fichier[colonnes_interet].dropna().copy()

#Clustering hdbcan
donnees_vitesse = fichier_ia[['SOG']].head(1000)
taille_min_cluster= 20

hdb_vitesse = hdbscan.HDBSCAN(min_cluster_size=taille_min_cluster)
hdb_vitesse.fit(donnees_vitesse)

#Ajout des labels
fichier_ia.loc[donnees_vitesse.index, 'cluster_vitesse'] = hdb_vitesse.labels_




C:\Users\fanny\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

C:\Users\fanny\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [ ]:
#hdbscan attribue un label -1 à tous les points isolés qui n’appartiennent à aucun cluster. 
# Donc on est obligés de filtré ces points. Ce sont surement des points en zone de transition entre deux orientation.

In [154]:
#calcul de l'indice de silhouette
from sklearn.metrics import silhouette_score

sil = silhouette_score(donnees_vitesse[hdb_vitesse.labels_ != -1], hdb_vitesse.labels_[hdb_vitesse.labels_ != -1])
sil=round(sil,3)
print("indice de silouhette pour min_cluster_size=",min_taille_clusters, ":",sil)

indice de silouhette pour min_cluster_size= 250 : 0.853


In [156]:

colonnes_multi = ['SOG', 'COG', 'Heading']
fichier_ia_multi = fichier[colonnes_multi].dropna().copy()
donnees_multi = fichier_ia_multi.head(1000)

# Clustering HDBSCAN
min_cluster_size = 20
hdb_multi = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size)
hdb_multi.fit(donnees_multi)

# Ajout des labels
fichier_ia_multi.loc[donnees_multi.index, 'cluster_multi'] = hdb_multi.labels_





C:\Users\fanny\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

C:\Users\fanny\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [ ]:

if len(np.unique(hdb_multi.labels_
[hdb_multi.labels_ != -1])) > 1:
    score_silhouette_multi = silhouette_score(donnees_multi[hdb_multi.labels_ != -1], hdb_multi.labels_
[hdb_multi.labels_ != -1])
    print("Silhouette Score (SOG + COG + heading) :", round(score_silhouette_multi, 3))
else:
    print("Pas assez de clusters valides pour calculer le silhouette score (multi-variables).")

Silhouette Score (SOG + COG + heading) : 0.657
